# Get Flashcards


In [ ]:
# !pip install tweepy wget python-dotenv

In [1]:
import tweepy 
import wget
import os
import dotenv

In [11]:
dotenv.load_dotenv(override=True)

True

In [12]:
# https://apps.twitter.com/
auth = tweepy.OAuthHandler(os.environ['consumer_key'], os.environ['consumer_secret'])
auth.set_access_token(os.environ['access_token'], os.environ['access_token_secret'])

In [14]:
api = tweepy.API(auth)

In [37]:
#Get 200 of Chris' tweet
tweets = api.user_timeline(
    screen_name = 'chrisalbon', 
    count = 300, 
    include_rts = False, 
    excludereplies = True
)

#200 isn't enough.  Keep getting tweets until we can't get anymore

In [38]:
last_id = tweets[-1].id
last_id

1066438696654659586

In [39]:
# Get more tweets: 300 tweets per loop
while (True):
    more_tweets = api.user_timeline(
        screen_name='chrisalbon',
        count=300,
        include_rts=False,
        exclude_replies=True,
        max_id=last_id-1
    )

    # There are no more tweets
    if (len(more_tweets) == 0):
        break
    else:
        last_id = more_tweets[-1].id - 1
        tweets = tweets + more_tweets
        
print(f'Num of tweets: {len(tweets)}')

Num of tweets: 252


In [40]:
# Check hashtags
[s.entities.get('hashtags') for s in tweets if s.entities.get('hashtags')]

[[{'text': 'devart', 'indices': [15, 22]}],
 [{'text': 'Forever', 'indices': [10, 18]}],
 [{'text': 'passion', 'indices': [85, 93]},
  {'text': 'honesty', 'indices': [94, 102]},
  {'text': 'lifestyle', 'indices': [103, 113]}],
 [{'text': 'NoYourCrying', 'indices': [101, 114]}]]

In [41]:
# Filter by those containing #machinelearningflashcards
# card_tweets = [s for s in tweets if 'https://t.co/eZ2bbpDzwV' in s.text]

def filter_card(s):
    urls = s.entities.get('urls')
    if urls:
        return(urls[0].get('display_url') == 'machinelearningflashcards.com')
    else:
        return False
card_tweets = list(filter(filter_card, tweets))
len(card_tweets)

22

In [27]:
# s = card_tweets[0]
# s.text.split(' http')[0]

In [42]:
media_files = dict()
for status in card_tweets:
    title = status.text.split(' http')[0]
    media = status.entities.get('media', [])
    if(len(media) > 0 and media[0]['type']=='photo' ):  #if tweet has media and media is photo
        media_files[title] = media[0]['media_url']  #get me the url

In [43]:
len(media_files)

21

In [44]:
media_files

{'Linear Discriminant Analysis For Dimensionality Reduction': 'http://pbs.twimg.com/media/Dt1S6GlVYAAb870.png',
 'How To Choose Hidden Unit Activation Functions': 'http://pbs.twimg.com/media/DtwMEaVU4AAX6-T.png',
 'Parameter Sharing': 'http://pbs.twimg.com/media/DtrFRatU8AArdJD.png',
 'Learning Rate': 'http://pbs.twimg.com/media/Dtl93DLU0AAJTH3.png',
 'Silhouette Coefficients': 'http://pbs.twimg.com/media/DtgukjFU4AAlwkQ.png',
 'Consistency': 'http://pbs.twimg.com/media/DtRVRYuVsAAw1xh.png',
 'Receiver Operating Characteristic': 'http://pbs.twimg.com/media/DtMFtz4UwAA0uKX.png',
 'Occams Razor': 'http://pbs.twimg.com/media/DtG8h-DUcAAI9vw.png',
 'Out-Of-Core': 'http://pbs.twimg.com/media/DtB0xATUUAALGnB.png',
 'Power Rule': 'http://pbs.twimg.com/media/Ds8m5-bU8AAfdDd.png',
 'Architecture Of A Neural Network': 'http://pbs.twimg.com/media/DrlMIGDU0AEWsRR.png',
 'Bagging Vs Dropout': 'http://pbs.twimg.com/media/DsoLeFbWwAEDPuG.png',
 'Feature Importance': 'http://pbs.twimg.com/media/Dsi83Y

## Write img to files

In [46]:
os.makedirs('ml-cards', exist_ok=True) # make a directory to store the photos in

for k, v in media_files.items():
    if not os.path.exists(f'ml-cards/{k}.png'):
        wget.download(v, out = f'ml-cards/{k}.png') # get the photos!